# DataPreparing

THe first step of our notebooks will be to run a DataPreparing script.  
This contains all the necessary code to transform our original images into images that are ready for AI training.  

To benefit from the perks of our Azure cloud service, we will be creating a new dataset to store our processed images.

Follow along in the HackMD documents to answer some questions introduced in these notebooks.

If you want more background information on these notebooks, the HackMD documents will help you as well.

## Setup

Our virtual machine might not have all packages installed yet. So let's go and install some packages.  
We can use cell-magic for this, which will allow us to stay inside this notebook and just executing the cells.  

Later on, these cells might nog be necessary anymore, which is why we include it at the top. During other builds, you can just ignore these.

As a best practice, let's make sure to only work on the version we know is safe. This is a great way to organising our AI projects. By keeping the versions linked like this, no unexpected new version would break our code!

In [ ]:
%%writefile requirements.txt

opencv-python==4.5.4.60

In [ ]:
# We want to make sure to use the right AzureML packages!
!/anaconda/envs/azureml_py38_PT_TF/bin/python -m pip install -r requirements.txt

In [ ]:
# This cell can be used to fill in some values that you will be referring to in the coming cells
train_test_split_factor = 0.20

In [ ]:
# Importing the default packages for data processing and visualisation
import numpy as np # Used to process our images in a data-format
import matplotlib.pyplot as plt # visualise the images
import cv2 # Process the images


import os
from glob import glob
import math

import warnings
warnings.filterwarnings("ignore") # Warnings that can be ignored will be ignored

import random
SEED = 42 # Everytime you want to randomize items, use this `random.seed(SEED)` option. This way, you are always having the same randomization as I have.
random.seed(SEED)

In [ ]:
# Import AzureML packages
from azureml.core import Workspace
from azureml.core import Dataset
from azureml.data.datapath import DataPath
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

## Step 1: Connecting to the Azure ML Workspace

Azure Machine Learning needs to connect through the Azure SDK with the Workspace object. This contains all the information inside of this 'Laboratory'

The information below should reflect your situation regarding Azure. You should have a ResourceGroup called '04_AzureML' and a workspace name called 'segersnathan' if you followed my instructions on HackMD.
The subscription ID, however, is something that has been created by Azure itself.

Luckily, this ML studio gives us a quick way to find this information.
Click on the \/-arrow in the upper-right corner over there ↗️, next to your profile picture.

Most of your information is in there as well, but you still can't find your subscription_**id** there ...

Press the 'Download config' option, and you'll be left with this information:

```json
{
    "subscription_id": "7c50f9c3-289b-4ae0-a075-08784b3b9042",
    "resource_group": "NathanReserve",
    "workspace_name": "segersnathan"
}
```

Which gives you exactly the information you need 🥰

There's also an option to use this configuration itself. Search for the documentation on how to do it: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py


In [ ]:
## Either get environment variables, or a fallback name, which is the second parameter.
## Currently, fill in the fallback values. Later on, we will make sure to work with Environment values. So we're already preparing for it in here!
workspace_name = os.environ.get('WORKSPACE', 'azure-ai-training')
subscription_id = os.environ.get('SUBSCRIPTION_ID', '763622cd-d9e1-46f1-84c7-635df9708641')
resource_group = os.environ.get('RESOURCE_GROUP', 'mlops-nathansegers')

In [ ]:
ws = Workspace.get(name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resource_group)

In [ ]:
ws

## Step 2 -- Data preparing

Let's get started preparing our data.
The original dataset contains images of Cats, Dogs and Pandas, but are not yet ready for AI processing.
They are all different shapes and sizes, so they need to be processed.

I could've done it earlier on and given you the processed data, but that's not really that fun, isn't it 😉

By performing these steps, you see how you can create datasets in a Programmatoric way, and upload new data.
We will make use of the Azure Machine Learning tools to easily manage all of our datasets in a structured way.

## Step 2.1 -- Checking our data

Let us first explore how the data looks. We'll create 3 subdirectories under a data directory, one for each animal.
If you want to update this to more animals later, simply adapt the `ANIMALS` list.

In [ ]:
ANIMALS = ['cats', 'dogs', 'pandas'] # As we have three classes, let's just defined them hoor. This way we can easily loop over them later.
# We can also work with ENUM's if you want another fun way of accessing properties:
# Feel free to uncomment these lines, I won't work with them, but you can always do so if yoi prefer


# from enum import Enum
# Animals = Enum('Animal', 'CATS DOGS PANDAS')

# print(list(Animals))
# print(Animals.CATS.name.lower()) # You can always get their effective string values this way

We will need to create temporary directories to store the images while we process them.
This script will create a `data` folder, and then make subdirectories for each animal.

In [ ]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)
for animal_name in ANIMALS:
    os.makedirs(os.path.join(data_folder, 'animals', animal_name), exist_ok=True)

In [ ]:
# Get all the datasets that were registered in the UI
# We can then easily select the ones we need
datasets = Dataset.get_all(workspace=ws) # Make sure to give our workspace with it
print(datasets)

### **TODO for students**

Write a check to see if we have our three datasets!!

In [ ]:
# Write your answer here

Uncomment the next cell to show a possible answer

In [ ]:
# %load code_solutions/CheckDatasets.txt

In [ ]:
from typing import List
def downloadTestImagesForAnimal(animal_name: str, amount_of_test_images: int) -> List[str]:
    """
    Download test images from an Azure dataset.
    Choose the amount of test images per animal you want.
    The images will be downloaded to the `data/<animal>` directory

    Parameters:
        animal_name (str): The name of the dataset / animal to download images from
        amount_of_test_images (int): The amount of test images to download
    Returns:
        file_paths (List[str]): list of file paths to display the images
    """
    animal_dataset = datasets[animal_name]
    test_images = animal_dataset.take(amount_of_test_images)
    file_paths = test_images.download(os.path.join(data_folder, 'animals', animal_name), overwrite=True)
    return file_paths

def displayTestImages(images: List[str]) -> None:
    fig, axes = plt.subplots(1, len(images), figsize=(12, 9), sharey=True)
    axes_1d = axes.ravel()
    for i in range(0, len(images)):    
        axes_1d[i].imshow(plt.imread(images[i]), cmap='gray')
        axes_1d[i].axis('off')
        
    plt.show()

Choose a number of images you want to show the users

In [ ]:
animal_images = {}
AMOUNT_OF_IMAGES = 5
for animal_name in ANIMALS:
    print(f"Downloading {AMOUNT_OF_IMAGES} images of {animal_name}...")
    animal_images[animal_name] = {}
    animal_images[animal_name]['images'] = downloadTestImagesForAnimal(animal_name, AMOUNT_OF_IMAGES)
    

In [ ]:
displayTestImages(animal_images['pandas']['images'])

### Step 2.2 Processing and uploading the resized images

Like we said in the introduction, we'll need to resize the images so they're all the same shape and size.
We are getting ready for a CNN model, which will require an input of 64, 64. You can go higher, but then it'll take longer to train, but also improve the accuracy.

In [ ]:
# Let's create a mounth point. Think of it like your D:/ drive on your PC
mount_path = os.path.join('/home/azureuser/cloudfiles/data')
os.makedirs(mount_path, exist_ok=True)

In [ ]:
# Defien a path to store the animal images onto. We'll choose for `data/processed/animals` this time. Again, create subdirectories for all the animals
processed_path = os.path.join(mount_path, 'processed', 'animals')
os.makedirs(processed_path, exist_ok=True)
for animal_name in ANIMALS:
    os.makedirs(os.path.join(processed_path, animal_name), exist_ok=True)

In [ ]:
# Defien a path to store the animal images onto. We'll choose for `data/processed/animals` this time. Again, create subdirectories for all the animals
original_path = os.path.join(mount_path, 'original', 'animals')
os.makedirs(original_path, exist_ok=True)
for animal_name in ANIMALS:
    os.makedirs(os.path.join(original_path, animal_name), exist_ok=True)

In [ ]:
!ls $mount_path/original/animals/cats

In [ ]:
def mountProcessAndUploadAnimalImages(animal_name):

    # The mount context is to load in the dataset to our directory.
    # Make sure to stop it when it's finished!

    # Get the dataset name for this animal, then mount to the directory
    mounted_context = datasets[animal_name].mount(os.path.join(mount_path, 'original', 'animals', animal_name))
    print('Starting the Mount context, to get all the original images.')
    mounted_context.start()

    # Get all the image paths with the `glob()` method.
    print(f'Resizing all images for {animal_name} ...')
    imagePaths = glob(f"{mount_path}/original/animals/{animal_name}/*.jpg")

    # Process all the images with OpenCV. Reading them, then resizing them to 64x64 and saving them once more.
    print(f"Processing {len(imagePaths)} images")
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (64, 64)) # Resize to a square of 64, 64
        cv2.imwrite(os.path.join(processed_path, animal_name, imagePath.split('/')[-1]), image)
    print(f'... done resizing. Stopping context now...')
    
    # Upload the directory as a new dataset
    print(f'Uploading directory now ...')
    resized_dataset = Dataset.File.upload_directory(
                        # Enter the sourece directory on our machine where the resized pictures are
                        src_dir = os.path.join(processed_path, animal_name),
                        # Create a DataPath reference where to store our images to. We'll use the default datastore for our workspace.
                        target = DataPath(datastore=ws.get_default_datastore(), path_on_datastore=f'processed_animals/{animal_name}'),
                        overwrite=True)
    # Make sure to register the dataset whenever everything is uploaded.
    resized_dataset.register(ws,
                            name=f'resized_{animal_name}',
                            description=f'{animal_name} images resized tot 64, 64',
                            tags={'animals': animal_name, 'AI-Model': 'CNN'}, # Optional tags, can always be interesting to keep track of these!
                            create_new_version=True)
    print(f'... Done')
    # Stop the context now.
    mounted_context.stop()
    print(f"... Context stopped and freed.")

In [ ]:
# Use this cell in case you need to unmount the context during an issue. this can happen whenever the 'mountProcessAndUploadAnimalImages' function fails with an error ...
#!sudo umount "/home/azureuser/cloudfiles/data/original/animals/cats"

In [ ]:
mountProcessAndUploadAnimalImages('cats')

In [ ]:
%%time
# Process all the animal images now. This can take a while!
# We'll use Cell magic once more, to time how long this takes!
# mountProcessAndUploadAnimalImages('cats')
mountProcessAndUploadAnimalImages('dogs')
mountProcessAndUploadAnimalImages('pandas')

Our Animal datasets are now registered onto the datasets of Azure, which is what we need. We can now safely delete all the images from our disk.

### **TODO for students**

Delete the images from disk in any way you want.


In [ ]:
# Write your answer here

Uncomment the next cell to see a possible answer.

In [ ]:
# %load code_solutions/RemoveProcessedDirectories.txt

## Step 2.3 -- Create Training and Testing splitted data

As you might remember from previous AI lessons, we want to split up our AI data in a batch of Training and Testing data, with the necessary labels.
We will take a portion of our data to train on, and the rest will be kept to test and validate our AI model.

For this, I chose a percentage in the first cell:
`train_test_split_factor = 0.20`

In [ ]:
training_datapaths = []
testing_datapaths = []
default_datastore = ws.get_default_datastore()
for animal_name in ANIMALS:
    # Get the dataset by name
    animal_dataset = Dataset.get_by_name(ws, f"resized_{animal_name}")

    # Get only the .JPG images
    animal_images = [img for img in animal_dataset.to_path() if img.split('.')[-1] == 'jpg']

    ## Concatenate the names for the animal_name and the img_path. Don't put a / between, because the img_path already contains that
    animal_images = [(default_datastore, f'processed_animals/{animal_name}{img_path}') for img_path in animal_images] # Make sure the paths are actual DataPaths
    
    random.seed(SEED) # Use the same random seed as I use and defined in the earlier cells
    random.shuffle(animal_images) # Shuffle the data so it's randomized
    
    ## Testing images
    amount_of_test_images = math.ceil(len(animal_images) * train_test_split_factor) # Get a small percentage of testing images

    animal_test_images = animal_images[:amount_of_test_images]
    animal_training_images = animal_images[amount_of_test_images:]
    
    # Add them all to the other ones
    testing_datapaths.extend(animal_test_images)
    training_datapaths.extend(animal_training_images)

### **TODO for students**

Write assertions to see if you have 600 testing images in total, and 200 for each animal.  
Update the values depending on your `train_test_split_factor` of course!

"What are Assertions?"
https://www.w3schools.com/python/ref_keyword_assert.asp

In [ ]:
# Write your answer here

Uncomment the cell below to find a possible answer.

In [ ]:
# %load code_solutions/Assertions.txt

### Step 2.4 -- Register Training and Testing Dataset

We don't need to re-upload our images, but we do need to register our Training and Testing datasets as well.

In [ ]:
testing_datapaths[:2]

In [ ]:
training_dataset = Dataset.File.from_files(path=training_datapaths)
testing_dataset = Dataset.File.from_files(path=testing_datapaths)

training_dataset = training_dataset.register(ws,
    name=f'animals-training-set',
    description=f'The Animal Images to train, resized tot 64, 64',
    tags={'animals': ','.join(ANIMALS), 'AI-Model': 'CNN', 'Split size': str(1 - train_test_split_factor), 'type': 'training'},
    create_new_version=True)

testing_dataset = testing_dataset.register(ws,
    name=f'animals-testing-set',
    description=f'The Animal Images to test, resized tot 64, 64',
    tags={'animals': ','.join(ANIMALS), 'AI-Model': 'CNN', 'Split size': str(train_test_split_factor), 'type': 'testing'},
    create_new_version=True)

## Finalizing

Now that we have all our datasets uploaded, we can continue to the next notebook where we will learn how to Train an AI model into Azure Machine Learning.

In [ ]:
# One last check for the dataset.
Dataset.get_all(ws)